In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from IPython.display import display
from data_cleaning import DataCleaning
from cleaner import DataCleaner
import pandas as pd

In [2]:
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 
data_cleaner = DataCleaning()

In [ ]:
#Legacy Store Details

In [3]:
#Extract
table_name = 'legacy_store_details'
store_df = data_extractor.read_rds_table(table_name)

#Clean
    
    #Numerical columns
#1. work through numeric column
#2. basic clean - ie remove any non-numerical value from column
#3. transform dt of the column (pd.tonumeric()) - opening_date starts here
#4. drop.na() - drops all NULLS

    #Non-numerical columns
#1. df.column_name.unique() - identify unique columns

#Example:
#clean_store_df = data_cleaning.clean_store_data(store_df)
print(type(store_df))

#View Data after cleaning
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(clean_store_df)



<class 'pandas.core.frame.DataFrame'>


In [ ]:
clean_store_df = data_cleaner.clean_store_data(store_df)
#print(clean_store_df)

#DataCleaner.clean_store_data(store_df)
#DataCleaner.hello('hi my name is sam')

#View Data after cleaning
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(clean_store_df)

In [ ]:
#Legacy Users

In [4]:
#Extract
table_name = 'legacy_users'
users_df = data_extractor.read_rds_table(table_name)
print('Legacy Users Table\n', users_df.head())
print('\n')

Legacy Users Table
    index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNe

In [ ]:
#Extract
table_name = 'orders_table'
orders_df = data_extractor.read_rds_table(table_name)
print('Orders Table Table\n', orders_df.head())
print('\n')

In [ ]:
table_name = 'legacy_store_details'
store_df = data_extractor.read_rds_table(table_name)
print('Legacy Store Details Table\n', store_df.head())
print('\n')

In [ ]:
table_name = 'legacy_users'
users_df = data_extractor.read_rds_table(table_name)
print('Legacy Users Table\n', users_df.head())
print('\n')

In [ ]:
#important code
import re
pattern = re.compile(r'^\D+$')
non_numeric_last_names = users_df[~users_df['last_name'].astype(str).str.match(pattern, na=False)]
display(non_numeric_last_names)